In [3]:
#I am setting this environment to practice and validate dataset and build something out of it, hopefully!!
#and for easier analaysis, ofcourse, VSCode is not Data Engineers friendly, HAHAHA!

In [4]:
import pandas as pd
import os
import datetime

In [77]:
staging_directory="../staging/"
bulk_csv_list=[ x for x in os.listdir(staging_directory) if 'bulk' in x ]
bulk_df_list=[pd.read_csv(staging_directory+file) for file in bulk_csv_list]
bulk_df=pd.concat(bulk_df_list, axis=0, ignore_index=True)

bulk_df.head()

,Date,Symbol,Security Name,Client Name,Buy/Sell,Quantity Traded,Trade Price / Wght. Avg. Price,Remarks
0,25-AUG-2023,ALPA,Alpa Laboratories Limited,GRAVITON RESEARCH CAPITAL LLP,BUY,133673,81.39,-
1,25-AUG-2023,AMBER,Amber Enterprises (I) Ltd,GOVERNMENT OF SINGAPORE,BUY,725239,2800.0,-
2,25-AUG-2023,ARIES,Aries Agro Limited,GRAVITON RESEARCH CAPITAL LLP,BUY,89917,193.17,-
3,25-AUG-2023,ASTRAMICRO,Astra Microwave Products,GRAVITON RESEARCH CAPITAL LLP,BUY,487675,386.36,-
4,25-AUG-2023,ATULAUTO,Atul Auto Limited,GRAVITON RESEARCH CAPITAL LLP,BUY,390565,457.74,-


In [79]:
#bulk Df is basically block deals happend by the client.
#These clients can be FII and DII. We will build something to identfy that later on.
#for now, I am trying to consolidate the client regime, how much of Indian stock they bought.
staging_directory="../staging/"
bulk_csv_list=[ x for x in os.listdir(staging_directory) if 'bulk' in x ]
bulk_df_list=[pd.read_csv(staging_directory+file) for file in bulk_csv_list]
bulk_df=pd.concat(bulk_df_list, axis=0, ignore_index=True)

#Schema changes: I am changing the schema as it is not good way to keep spaces in between.
#let's create a standard of camel casing i.e Quantity Traded -> QuantityTraded.
#Also, Let's Fix the Date Standards. 25-Aug-2023 -> 2023-08-25

new_schema={'Symbol':'SecuritiesCode','Client Name':'ClientName','Quantity Traded':'QuantityTraded', 'Trade Price / Wght. Avg. Price':'MeanWghtPrice', 'Buy/Sell':'Transaction'}
bulk_df_cleaned=bulk_df.rename(columns=new_schema).drop(columns=['Security Name','Remarks']).drop_duplicates()
bulk_df_cleaned['Date']=pd.to_datetime(bulk_df_cleaned.Date).dt.strftime('%Y-%m-%d')

#Negating QuantityTraded for Sell Transaction type to get the net Transaction.
#Net QunatityTraded
# - Positive -> Buy
# - Negative -> Sell
def negate_quantityTraded(Transaction,QuantityTraded):
    if Transaction == "SELL":
        return -1*QuantityTraded
    else:
        return QuantityTraded

bulk_df_cleaned['QuantityTraded']=bulk_df_cleaned.apply(lambda x: negate_quantityTraded(x['Transaction'],x['QuantityTraded']),axis=1)
bulk_df_cleaned.sort_values(by=['ClientName'],ascending=False).head(20)

#Possible case:
#   -  Now, we don't know how much shares these Client posses. Anyways we are concerned with Change.
#   -  There could be case of Intraday Trading i.e Buyer will buy/sell or sell/buy in same day, net buy=0
#   -  long term i.e CNC buy.

/tmp/ipykernel_4647/2887028183.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bulk_df_cleaned['Date']=pd.to_datetime(bulk_df_cleaned.Date).dt.strftime('%Y-%m-%d')


,Date,SecuritiesCode,ClientName,Transaction,QuantityTraded,MeanWghtPrice
843,2023-03-24,ZYDUSWELL,ZYDUS FAMILY TRUST,BUY,510272,"1,499.67"
6122,2023-08-07,ZYDUSWELL,ZYDUS FAMILY TRUST,BUY,323868,"1,460.00"
6584,2023-08-14,JAIPURKURT,ZYANA STOCKS AND COMMODITIES,SELL,-100000,89.00
6555,2023-08-11,VINSYS,ZINNIA GLOBAL FUND PCC - CELL DEWCAP FUND,BUY,100000,217.60
5388,2023-07-24,VERTEXPLUS,ZINNIA GLOBAL FUND PCC - BLUE JADE INVESTMENT ...,SELL,-30000,132.00
4560,2023-07-07,VERTEXPLUS,ZINNIA GLOBAL FUND PCC - BLUE JADE INVESTMENT ...,SELL,-60000,109.00
4637,2023-07-10,VERTEXPLUS,ZINNIA GLOBAL FUND PCC - BLUE JADE INVESTMENT ...,SELL,-30000,107.00
684,2023-03-20,VIAZ,ZENITH MEDICINE PRIVATE LIMITED,BUY,160000,55.49
3928,2023-06-23,KSHITIJPOL,ZENAB AIYUB YACOOBALI,BUY,320390,8.28
3646,2023-06-19,KSHITIJPOL,ZENAB AIYUB YACOOBALI,SELL,-281330,8.70


In [85]:
#now that we have some cleaning done, let check the regime of Client.
#Regime of Client: Varied Stocks that single client has bought.
#PS: you will see so many reassignments, coz I hate using more variables.

ClientPortfolio=bulk_df_cleaned
tdf=ClientPortfolio.groupby(['ClientName','Date','SecuritiesCode'])[['QuantityTraded']].sum().reset_index()

#TradeType: Indicates type of Trade occurred depending on Net QuantityTraded.
#if Net QunatityTraded is 0 -> Intraday Trading else or else we will consider it long term.
def determine_trade_type(NetQunatityTrade) -> str :
    if NetQunatityTrade == 0:
        return "INTRADAY"
    elif NetQunatityTrade > 0:
        return "BUY"
    else :
        return "SELL"

tdf['TradeType']=tdf.apply(lambda x: determine_trade_type(x['QuantityTraded']), axis=1 )
#Filter Intraday data
tdf_Intraday=tdf[ tdf['TradeType']=='INTRADAY']
tdf_other=tdf[ tdf['TradeType']!='INTRADAY']
tdf_Intraday.to_csv("../data/IntradayData.csv")
tdf_other.to_csv("../data/ClientPortFolio.csv")

#I am debating wether to filter out Intraday TradeType for now or not?
#But Keeping might give us better insight on which Stocks are good for intraday

tdf_other.head(20)
#I know this will only give us the Client who bought most number of stocks in a day, which it bought that is still with bulk_df_cleaned
#I have alogorithm to store all the stocks bought in fixed lenght, will implement it later on. 
# Just for guess, it has to do with bit map. HAHA!


,ClientName,Date,SecuritiesCode,QuantityTraded,TradeType
0,2015 GROVER FAMILY TRUST,2023-03-28,63MOONS,-300000,SELL
1,360 ONE MUTUL FUND,2023-06-09,CMSINFO,1200000,BUY
2,3P INDIA EQUITY FUND 1,2023-06-23,LANDMARK,304000,BUY
3,A AND S TRADELINK,2023-04-03,VIVIMEDLAB,712573,BUY
4,A S CONFIN PRIVATE LIMITED,2023-07-25,DIL,18465,BUY
5,A S CONFIN PRIVATE LIMITED,2023-07-26,DIL,25123,BUY
6,A S CONFIN PRIVATE LIMITED,2023-07-27,DIL,19263,BUY
7,A S CONFIN PRIVATE LIMITED,2023-07-28,DIL,320763,BUY
8,A S CONFIN PRIVATE LIMITED,2023-08-01,DIL,-383276,SELL
9,A S CONFIN PRIVATE LIMITED,2023-08-02,DIL,1481588,BUY
